In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string

In [2]:
letters = list(string.ascii_lowercase)

In [88]:
fi = pd.read_json('../Data/fighter/fighters-a.json')
for letter in letters:
    if letter != 'a':
        data = pd.read_json(f'../Data/fighter/fighters-{letter}.json')
        fi = fi.append(data, ignore_index=True, )

In [89]:
fighters = pd.read_csv('../Data/ufc_fighters.csv')
matches = pd.read_csv('../Data/ufc_bouts.csv', index_col='bout_id')

In [90]:
fi.sample(3)

,fname,lname,nname,height,weight,reach,stance,win,lose,draw
841,Oleksandr,Doskalchuk,None,"5' 6""",125 lbs.,--,Southpaw,10,2,0
2459,Saimon,Oliveira,None,"5' 4""",135 lbs.,"72.0""",Orthodox,18,3,0
10,Scott,Adams,None,"6' 0""",225 lbs.,--,Southpaw,8,1,0


In [91]:
fi.height.nunique()

27

In [92]:
fighters.head(2)

,fighter_id,name,win,lose,draw,nc,height,weight,reach,stance,dob,SLpM,Str_Acc,SApM,Str_Def,TD_Avg,TD_Acc,TD_Def,Sub_Avg,last_updated
0,1,Luciano Azevedo,16,9,1,0,"6' 3""",161 lbs.,NaN,Orthodox,"Jun 25, 1981",0.76,45%,1.97,27%,2.28,11%,0%,0.0,2018-11-21 19:35:04.893952
1,2,Michael Byrnes,5,2,0,0,"5' 11""",155 lbs.,NaN,NaN,"Jul 17, 1990",0.00,0%,0.00,0%,0.00,0%,0%,0.0,2018-11-21 19:35:09.304165


In [31]:
matches.head(2)

,event_name,date,location,attendance,result,fighter1,fighter2,winner,weight_class,title_fight,method,end_round,end_time
bout_id,,,,,,,,,,,,,
1,UFC Fight Night: Magny vs. Ponzinibbio,2018-11-17,"Buenos Aires, Buenos Aires, Argentina",10245.0,win,Santiago Ponzinibbio,Neil Magny,Santiago Ponzinibbio,Welterweight,f,KO/TKO-Punch,4,2:36
2,UFC Fight Night: Magny vs. Ponzinibbio,2018-11-17,"Buenos Aires, Buenos Aires, Argentina",10245.0,win,Ricardo Lamas,Darren Elkins,Ricardo Lamas,Featherweight,f,KO/TKO-Elbows,3,4:09


In [32]:
matches.isna().sum()

event_name        0
date              0
location          0
attendance      196
result            0
fighter1          0
fighter2          0
winner           82
weight_class      0
title_fight       0
method            0
end_round         0
end_time          0
dtype: int64

In [33]:
matches.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4878 entries, 1 to 4878
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   event_name    4878 non-null   object 
 1   date          4878 non-null   object 
 2   location      4878 non-null   object 
 3   attendance    4682 non-null   float64
 4   result        4878 non-null   object 
 5   fighter1      4878 non-null   object 
 6   fighter2      4878 non-null   object 
 7   winner        4796 non-null   object 
 8   weight_class  4878 non-null   object 
 9   title_fight   4878 non-null   object 
 10  method        4878 non-null   object 
 11  end_round     4878 non-null   int64  
 12  end_time      4878 non-null   object 
dtypes: float64(1), int64(1), object(11)
memory usage: 533.5+ KB


## Cleaning the matches data

In [72]:
matches = matches.loc[matches["result"] == "win"].copy()

In [73]:
matches['location'] = matches['location'].map(lambda x: x.split(',')[-1])

In [74]:
matches['figth_year'] = matches['date'].map(lambda x: x.split('-')[0])

In [75]:
matches['method'] = matches['method'].map(lambda x: x.split('-')[0])

In [76]:
matches['attended'] = matches['attendance ']

In [77]:
matches['fight_coutry'] = matches['location']

In [78]:
matches.drop(["event_name", "date", "location", "attendance ", "result", "end_time"],
          axis=1, inplace=True)

In [79]:
matches.head(2)

,fighter1,fighter2,winner,weight_class,title_fight,method,end_round,figth_year,attended,fight_coutry
bout_id,,,,,,,,,,
1,Santiago Ponzinibbio,Neil Magny,Santiago Ponzinibbio,Welterweight,f,KO/TKO,4,2018,10245.0,Argentina
2,Ricardo Lamas,Darren Elkins,Ricardo Lamas,Featherweight,f,KO/TKO,3,2018,10245.0,Argentina


In [80]:
# winner is always fighter 1 
matches["fighter1"].equals(matches["winner"])

True

In [81]:
matches[matches["fighter1"] != matches['winner']]

,fighter1,fighter2,winner,weight_class,title_fight,method,end_round,figth_year,attended,fight_coutry
bout_id,,,,,,,,,,


In [82]:
# randomly swap fighter1 and fighter2 for half of the dataset to create "negative cases"
swap_indices = np.random.choice(len(matches), size = len(matches) // 2, replace = False)
matches.iloc[swap_indices, [0, 1]] = matches.iloc[swap_indices, [1, 0]].values

In [83]:
matches["winner"] = matches["winner"] == matches["fighter1"]
matches["winner"] = matches["winner"].astype(int)
matches["winner"].value_counts()

0    2398
1    2398
Name: winner, dtype: int64

In [84]:
matches

,fighter1,fighter2,winner,weight_class,title_fight,method,end_round,figth_year,attended,fight_coutry
bout_id,,,,,,,,,,
1,Santiago Ponzinibbio,Neil Magny,1,Welterweight,f,KO/TKO,4,2018,10245.0,Argentina
2,Darren Elkins,Ricardo Lamas,0,Featherweight,f,KO/TKO,3,2018,10245.0,Argentina
3,Johnny Walker,Khalil Rountree Jr.,1,Light Heavyweight,f,KO/TKO,1,2018,10245.0,Argentina
4,Cezar Ferreira,Ian Heinisch,0,Middleweight,f,DEC,3,2018,10245.0,Argentina
5,Marlon Vera,Guido Cannetti,1,Bantamweight,f,SUB,2,2018,10245.0,Argentina
...,...,...,...,...,...,...,...,...,...,...
4874,Chas Skelly,Bobby Moffett,0,Featherweight,f,SUB,2,2018,11426.0,USA
4875,Davi Ramos,John Gunther,1,Lightweight,f,SUB,1,2018,11426.0,USA
4876,Julian Erosa,Devonte Smith,0,Lightweight,f,KO/TKO,1,2018,11426.0,USA


In [85]:
matches[matches["fighter1"]=="Dong Hyun Kim"]

,fighter1,fighter2,winner,weight_class,title_fight,method,end_round,figth_year,attended,fight_coutry
bout_id,,,,,,,,,,
144,Dong Hyun Kim,Takanori Gomi,1,Lightweight,f,KO/TKO,1,2017,8571.0,Japan
1136,Dong Hyun Kim,Jason Tan,1,Welterweight,f,KO/TKO,3,2008,14773.0,USA
1193,Dong Hyun Kim,Matt Brown,1,Welterweight,f,DEC,3,2008,14736.0,USA
1382,Dong Hyun Kim,TJ Grant,1,Welterweight,f,DEC,3,2009,10871.0,USA
1582,Dong Hyun Kim,Amir Sadollah,1,Welterweight,f,DEC,3,2010,14996.0,USA
1739,Dong Hyun Kim,Nate Diaz,1,Welterweight,f,DEC,3,2011,12874.0,USA
2023,Dong Hyun Kim,Sean Pierson,1,Welterweight,f,DEC,3,2011,NaN,USA
2217,Dong Hyun Kim,Demian Maia,0,Welterweight,f,KO/TKO,1,2012,15104.0,USA
2314,Dong Hyun Kim,Paulo Thiago,1,Welterweight,f,DEC,3,2012,8415.0,China


In [86]:
# again, handel duplicate names
matches_clean = matches.copy()
for col in ["fighter1", "fighter2"]:
    matches_clean.loc[(matches_clean[col] == "Michael McDonald") &
                    (matches_clean["weight_class"] == "Light Heavyweight"), col] = "Michael McDonald 205"
    
    matches_clean.loc[(matches_clean[col] == "Dong Hyun Kim") &
                    (matches_clean["weight_class"] == "Lightweight"), col]= "Dong Hyun Kim 155"
    
    matches_clean.loc[(matches_clean[col] == "Tony Johnson") &
                    (matches_clean["weight_class"] == "Middleweight"), col] = "Tony Johnson 185"
    
    matches_clean.loc[(matches_clean[col] == "Mike Davis") &
                    (matches_clean["weight_class"] == "Featherweight"), col] = "Mike Davis 145"

In [87]:
matches_clean[matches_clean["fighter1"] == "Dong Hyun Kim 155"]

,fighter1,fighter2,winner,weight_class,title_fight,method,end_round,figth_year,attended,fight_coutry
bout_id,,,,,,,,,,
144,Dong Hyun Kim 155,Takanori Gomi,1,Lightweight,f,KO/TKO,1,2017,8571.0,Japan
3924,Dong Hyun Kim 155,Polo Reyes,0,Lightweight,f,KO/TKO,3,2016,15587.0,USA
4542,Dong Hyun Kim 155,Damien Brown,1,Lightweight,f,DEC,3,2018,12437.0,Australia


## Cleaning fighters data

In [94]:
# check if there are fighters with the same name
fighters[fighters.duplicated(subset="name", keep=False)]

,fighter_id,name,win,lose,draw,nc,height,weight,reach,stance,dob,SLpM,Str_Acc,SApM,Str_Def,TD_Avg,TD_Acc,TD_Def,Sub_Avg,last_updated
511,512,Michael McDonald,17,4,0,0,"5' 9""",135 lbs.,"70""",Orthodox,"Jan 15, 1991",2.69,42%,2.76,57%,1.09,66%,52%,1.4,2018-11-21 20:06:32.497948
513,514,Michael McDonald,1,1,0,0,"5' 11""",205 lbs.,NaN,Orthodox,"Feb 06, 1965",0.00,0%,0.40,50%,0.00,0%,0%,0.0,2018-11-21 20:06:39.825372
845,845,Dong Hyun Kim,16,8,3,0,"5' 11""",155 lbs.,"70""",Orthodox,"Sep 09, 1988",3.77,45%,4.53,53%,1.67,60%,33%,0.0,2018-11-21 20:26:57.805101
852,852,Dong Hyun Kim,22,4,1,1,"6' 2""",170 lbs.,"76""",Southpaw,"Nov 17, 1981",2.12,49%,1.89,58%,2.93,43%,71%,0.6,2018-11-21 20:27:20.82459
946,947,Tony Johnson,11,3,0,0,"6' 1""",265 lbs.,NaN,NaN,NaN,2.00,53%,4.73,31%,2.00,22%,0%,0.0,2018-11-21 20:33:09.727428
954,955,Tony Johnson,4,2,0,0,NaN,185 lbs.,NaN,NaN,NaN,0.00,0%,0.00,0%,0.00,0%,0%,0.0,2018-11-21 20:33:41.289451
1674,1674,Mike Davis,5,1,0,0,"6' 0""",145 lbs.,"72""",Orthodox,"Oct 07, 1992",5.13,49%,7.40,48%,0.00,0%,100%,0.0,2018-11-21 21:17:43.856431
1679,1679,Mike Davis,2,0,0,0,NaN,NaN,NaN,NaN,NaN,0.00,0%,0.00,0%,0.00,0%,0%,0.0,2018-11-21 21:18:02.161503


In [95]:
fighters.iloc[513, 1] = "Michael McDonald 205"
fighters.iloc[845, 1] = "Dong Hyun Kim 155"
fighters.iloc[954, 1] = "Tony Johnson 185"
fighters.iloc[1674, 1] = "Mike Davis 145"

In [96]:
# use fighter names as index
fighters.drop("fighter_id", axis=1, inplace=True)
fighters.set_index("name", inplace=True)

In [97]:
# Some fighters do not have statistics available, and we will remove those fighters.
fighers_clean = fighters.loc[~((fighters["SLpM"] == 0) &
                               (fighters["Str_Acc"] == "0%") & 
                               (fighters["SApM"] == 0) &
                               (fighters["Str_Def"] == "0%") &
                               (fighters["TD_Avg"] == 0) &
                               (fighters["TD_Acc"] == "0%") &
                               (fighters["TD_Def"] == "0%") &
                               (fighters["Sub_Avg"] == 0))].copy()

In [98]:
print("{0} fighers in total, after clean up: {1} fighers".format(len(fighters), len(fighers_clean)))

3187 fighers in total, after clean up: 2559 fighers


In [99]:
# add winning percentages
fighers_clean["win%"] = 100 * (fighers_clean["win"] / (fighers_clean["win"] +
                                                 fighers_clean["lose"] +
                                                 fighers_clean["draw"] +
                                                 fighers_clean["nc"]))
# change datatypes
percentages = ["Str_Acc", "Str_Def", "TD_Acc", "TD_Def"]
statistics = ["SLpM", "Str_Acc", "SApM", "Str_Def", "TD_Avg", "TD_Acc", "TD_Def", "Sub_Avg", "win%"]

fighers_clean.loc[:, percentages] = fighers_clean.loc[:, percentages].applymap(
    lambda x: x.replace("%", ""))

fighers_clean.loc[:, statistics] = fighers_clean.loc[:, statistics].astype(np.float32)

In [100]:
fighers_clean.head()

,win,lose,draw,nc,height,weight,reach,stance,dob,SLpM,Str_Acc,SApM,Str_Def,TD_Avg,TD_Acc,TD_Def,Sub_Avg,last_updated,win%
name,,,,,,,,,,,,,,,,,,,
Luciano Azevedo,16,9,1,0,"6' 3""",161 lbs.,NaN,Orthodox,"Jun 25, 1981",0.76,45.0,1.97,27.0,2.28,11.0,0.0,0.0,2018-11-21 19:35:04.893952,61.53846
Gleidson Cutis,7,3,0,0,"5' 9""",155 lbs.,NaN,Orthodox,"Feb 07, 1989",2.99,52.0,8.28,59.0,0.00,0.0,0.0,0.0,2018-11-21 19:35:11.211072,70.00000
Rolando Dy,9,7,1,1,"5' 8""",145 lbs.,"69""",Orthodox,"Aug 11, 1990",3.04,37.0,4.47,52.0,0.30,20.0,68.0,0.0,2018-11-21 19:35:14.677808,50.00000
Jessica Eye,13,6,0,1,"5' 6""",125 lbs.,"66""",Orthodox,"Jul 27, 1986",3.42,36.0,3.32,59.0,0.59,71.0,60.0,0.8,2018-11-21 19:35:17.495278,65.00000
Katsuaki Furuki,1,1,0,0,"6' 0""",168 lbs.,NaN,NaN,"Nov 10, 1980",0.27,10.0,2.67,39.0,0.00,0.0,0.0,3.0,2018-11-21 19:35:21.053769,50.00000


In [101]:
# use born year only
def get_year(dob):
    if pd.isna(dob):
        return "unk"
    return dob.split(", ")[-1]

fighers_clean["born_year"] = fighers_clean["dob"].map(lambda dob: get_year(dob))

In [102]:
fighers_clean.head()

,win,lose,draw,nc,height,weight,reach,stance,dob,SLpM,Str_Acc,SApM,Str_Def,TD_Avg,TD_Acc,TD_Def,Sub_Avg,last_updated,win%,born_year
name,,,,,,,,,,,,,,,,,,,,
Luciano Azevedo,16,9,1,0,"6' 3""",161 lbs.,NaN,Orthodox,"Jun 25, 1981",0.76,45.0,1.97,27.0,2.28,11.0,0.0,0.0,2018-11-21 19:35:04.893952,61.53846,1981
Gleidson Cutis,7,3,0,0,"5' 9""",155 lbs.,NaN,Orthodox,"Feb 07, 1989",2.99,52.0,8.28,59.0,0.00,0.0,0.0,0.0,2018-11-21 19:35:11.211072,70.00000,1989
Rolando Dy,9,7,1,1,"5' 8""",145 lbs.,"69""",Orthodox,"Aug 11, 1990",3.04,37.0,4.47,52.0,0.30,20.0,68.0,0.0,2018-11-21 19:35:14.677808,50.00000,1990
Jessica Eye,13,6,0,1,"5' 6""",125 lbs.,"66""",Orthodox,"Jul 27, 1986",3.42,36.0,3.32,59.0,0.59,71.0,60.0,0.8,2018-11-21 19:35:17.495278,65.00000,1986
Katsuaki Furuki,1,1,0,0,"6' 0""",168 lbs.,NaN,NaN,"Nov 10, 1980",0.27,10.0,2.67,39.0,0.00,0.0,0.0,3.0,2018-11-21 19:35:21.053769,50.00000,1980


In [103]:
# remove unused columns
fighers_clean.drop(["weight", "dob", "last_updated"], inplace=True, axis=1)

In [104]:
fighers_clean.head()

,win,lose,draw,nc,height,reach,stance,SLpM,Str_Acc,SApM,Str_Def,TD_Avg,TD_Acc,TD_Def,Sub_Avg,win%,born_year
name,,,,,,,,,,,,,,,,,
Luciano Azevedo,16,9,1,0,"6' 3""",NaN,Orthodox,0.76,45.0,1.97,27.0,2.28,11.0,0.0,0.0,61.53846,1981
Gleidson Cutis,7,3,0,0,"5' 9""",NaN,Orthodox,2.99,52.0,8.28,59.0,0.00,0.0,0.0,0.0,70.00000,1989
Rolando Dy,9,7,1,1,"5' 8""","69""",Orthodox,3.04,37.0,4.47,52.0,0.30,20.0,68.0,0.0,50.00000,1990
Jessica Eye,13,6,0,1,"5' 6""","66""",Orthodox,3.42,36.0,3.32,59.0,0.59,71.0,60.0,0.8,65.00000,1986
Katsuaki Furuki,1,1,0,0,"6' 0""",NaN,NaN,0.27,10.0,2.67,39.0,0.00,0.0,0.0,3.0,50.00000,1980


## Combining both data

In [106]:
all_fighter_names = fighers_clean.index.values.tolist()

matches_clean = matches_clean.loc[(matches_clean["fighter1"].isin(all_fighter_names)) &
                              (matches_clean["fighter2"].isin(all_fighter_names))]

In [107]:
matches_clean.reset_index(inplace=True, drop=True)

In [109]:
matches_clean.head()

,fighter1,fighter2,winner,weight_class,title_fight,method,end_round,figth_year,attended,fight_coutry
0,Santiago Ponzinibbio,Neil Magny,1,Welterweight,f,KO/TKO,4,2018,10245.0,Argentina
1,Darren Elkins,Ricardo Lamas,0,Featherweight,f,KO/TKO,3,2018,10245.0,Argentina
2,Johnny Walker,Khalil Rountree Jr.,1,Light Heavyweight,f,KO/TKO,1,2018,10245.0,Argentina
3,Cezar Ferreira,Ian Heinisch,0,Middleweight,f,DEC,3,2018,10245.0,Argentina
4,Marlon Vera,Guido Cannetti,1,Bantamweight,f,SUB,2,2018,10245.0,Argentina


In [110]:
fighter1_data = fighers_clean.loc[matches_clean["fighter1"]]
fighter1_data = fighter1_data.add_suffix('_fighter1')
fighter2_data = fighers_clean.loc[matches_clean["fighter2"]]
fighter2_data = fighter2_data.add_suffix('_fighter2')

In [111]:
fighter1_data.head()

,win_fighter1,lose_fighter1,draw_fighter1,nc_fighter1,height_fighter1,reach_fighter1,stance_fighter1,SLpM_fighter1,Str_Acc_fighter1,SApM_fighter1,Str_Def_fighter1,TD_Avg_fighter1,TD_Acc_fighter1,TD_Def_fighter1,Sub_Avg_fighter1,win%_fighter1,born_year_fighter1
name,,,,,,,,,,,,,,,,,
Santiago Ponzinibbio,28,3,0,0,"6' 0""","73""",Orthodox,4.27,39.0,4.02,64.0,0.54,44.0,60.0,0.0,90.322578,1986
Darren Elkins,25,7,0,0,"5' 10""","71""",Orthodox,3.36,37.0,2.83,53.0,2.68,35.0,57.0,1.3,78.125000,1984
Johnny Walker,15,3,0,0,"6' 5""","82""",Orthodox,5.37,70.0,3.36,25.0,0.89,100.0,100.0,2.6,83.333336,1992
Cezar Ferreira,14,7,0,0,"6' 1""","78""",Southpaw,1.83,42.0,2.14,55.0,2.97,59.0,90.0,0.5,66.666664,1985
Marlon Vera,12,5,1,0,"5' 8""","70""",Switch,3.23,44.0,3.93,52.0,0.78,37.0,68.0,1.3,66.666664,1992


In [112]:
fighter2_data.head()

,win_fighter2,lose_fighter2,draw_fighter2,nc_fighter2,height_fighter2,reach_fighter2,stance_fighter2,SLpM_fighter2,Str_Acc_fighter2,SApM_fighter2,Str_Def_fighter2,TD_Avg_fighter2,TD_Acc_fighter2,TD_Def_fighter2,Sub_Avg_fighter2,win%_fighter2,born_year_fighter2
name,,,,,,,,,,,,,,,,,
Neil Magny,21,8,0,0,"6' 3""","80""",Orthodox,3.86,46.0,2.22,56.0,2.62,46.0,60.0,0.3,72.413795,1987
Ricardo Lamas,19,7,0,0,"5' 8""","71""",Orthodox,2.87,45.0,2.61,59.0,1.66,32.0,47.0,1.0,73.076920,1982
Khalil Rountree Jr.,8,2,0,2,"6' 1""","76""",Southpaw,2.30,34.0,3.29,38.0,0.00,0.0,50.0,0.3,66.666664,1990
Ian Heinisch,12,1,0,0,"5' 11""",NaN,Orthodox,3.49,58.0,1.67,52.0,0.81,14.0,50.0,1.6,92.307693,1988
Guido Cannetti,8,5,0,0,"5' 6""","68""",Southpaw,2.80,52.0,1.95,51.0,2.80,50.0,75.0,0.3,61.538460,1979


In [113]:
len(fighter1_data), len(fighter2_data)

(4591, 4591)

In [114]:
fighter1_data.reset_index(inplace=True, drop=True)
fighter2_data.reset_index(inplace=True, drop=True)
combined = pd.concat([matches_clean, fighter1_data, fighter2_data], axis=1, sort=False)

In [115]:
combined.head()

,fighter1,fighter2,winner,weight_class,title_fight,method,end_round,figth_year,attended,fight_coutry,...,SLpM_fighter2,Str_Acc_fighter2,SApM_fighter2,Str_Def_fighter2,TD_Avg_fighter2,TD_Acc_fighter2,TD_Def_fighter2,Sub_Avg_fighter2,win%_fighter2,born_year_fighter2
0,Santiago Ponzinibbio,Neil Magny,1,Welterweight,f,KO/TKO,4,2018,10245.0,Argentina,...,3.86,46.0,2.22,56.0,2.62,46.0,60.0,0.3,72.413795,1987
1,Darren Elkins,Ricardo Lamas,0,Featherweight,f,KO/TKO,3,2018,10245.0,Argentina,...,2.87,45.0,2.61,59.0,1.66,32.0,47.0,1.0,73.076920,1982
2,Johnny Walker,Khalil Rountree Jr.,1,Light Heavyweight,f,KO/TKO,1,2018,10245.0,Argentina,...,2.30,34.0,3.29,38.0,0.00,0.0,50.0,0.3,66.666664,1990
3,Cezar Ferreira,Ian Heinisch,0,Middleweight,f,DEC,3,2018,10245.0,Argentina,...,3.49,58.0,1.67,52.0,0.81,14.0,50.0,1.6,92.307693,1988
4,Marlon Vera,Guido Cannetti,1,Bantamweight,f,SUB,2,2018,10245.0,Argentina,...,2.80,52.0,1.95,51.0,2.80,50.0,75.0,0.3,61.538460,1979


In [116]:
len(combined)

4591

In [117]:
combined.to_csv("../Data/prediction_data.csv")

In [118]:
fi.to_csv("../Data/fighters_scraped.csv")